# ARIMA (package [`sktime`](https://www.sktime.net/en/stable/))

Specification of ARIMA(p,d,q) (ARIMA = AutoRegressive Integrated Moving Average)

$$
\begin{aligned}
	\Delta ^d y_t &= (\alpha_0+\alpha_1 t)+\sum_{j=1}^p\phi_j\Delta^d y_{t-j}+u_t+\sum_{s=1}^q\theta_s u_{t-s} &
	u_t&\sim WN(0,\sigma^2)
\end{aligned}
$$
where
* p is an oder of autoregressive part
* d is an integration order 
* q is an order of moving averageп part
* $\alpha_0$ is an intercept/cons (d=0) or drift (d>0)

Specification by means of  lag operator

$$
\begin{aligned}
	\phi_p(L)(1-L)^dy_t&=(\alpha_0+\alpha_1t)+\theta_q(L)u_t & u_t&\sim WN(0,\sigma^2)
\end{aligned}
$$
with polynomials
$$
\begin{aligned}
	\phi_p(z)&=1-\phi_1z-\cdots-\phi_pz^p & \theta_q(z)&=1+\theta_1z+\cdots+\theta_qz^q
\end{aligned}
$$

In [ ]:
import numpy as np
import pandas as pd

from sktime.forecasting.arima import ARIMA, AutoARIMA
# from sktime.forecasting.arima import StatsModelsARIMA
# from sktime.forecasting.statsforecast import StatsForecastAutoARIMA
from sktime.utils.plotting import plot_series
# временной горизонт для прогнозирования
from sktime.forecasting.base import ForecastingHorizon

import pandas_datareader.data as web

# тесты диагностики
from statsmodels.stats.diagnostic import het_arch, acorr_ljungbox

# настройки визуализации
import matplotlib.pyplot as plt

# Не показывать Warnings
import warnings
warnings.simplefilter(action='ignore', category=Warning)
# Не показывать ValueWarning, ConvergenceWarning из statsmodels
from statsmodels.tools.sm_exceptions import ValueWarning, ConvergenceWarning
warnings.simplefilter('ignore', category=ValueWarning)
warnings.simplefilter('ignore', category=ConvergenceWarning)

For fitting we use class (in fact from package `pmdarima`)
* [ARIMA](https://www.sktime.net/en/stable/api_reference/auto_generated/sktime.forecasting.arima.ARIMA.html): model of given order
* [AutoARIMA](https://www.sktime.net/en/stable/api_reference/auto_generated/sktime.forecasting.arima.AutoARIMA.html): fitting of "optimal order"

We need to set parameters `order` (model's order) and  `trend`

|Parameter|`trend`|
|-|-|
|$\alpha_0=\alpha_1=0$|'n'|
|$\alpha_0\ne0,\alpha_1=0$|'c'|
|$\alpha_0,\alpha_1\ne0$|'ct'|

Alternatively we can use classes
* [StatsModelsARIMA](https://www.sktime.net/en/stable/api_reference/auto_generated/sktime.forecasting.arima.StatsModelsARIMA.html): model of given order (from `statsmodels`)
* [StatsForecastAutoARIMA](https://www.sktime.net/en/stable/api_reference/auto_generated/sktime.forecasting.statsforecast.StatsForecastAutoARIMA.html): fitting of "optimal order" (from `statsforecast`)



## Fitting of ARIMA of given order

Let's import from [`FRED`](https://fred.stlouisfed.org/) weekly data on 3-Month Treasury Bill Secondary Market Rate (Symbol [`WTB3MS`](https://fred.stlouisfed.org/series/WTB3MS)) rom 2000-01-01 to 2023-12-31 as `y` DataFrame

In [ ]:
y = web.DataReader(name='WTB3MS', data_source='fred', start='2000-01-01', end='2023-12-31')

Consider ARIMA(2,1,1) without drift for series `y`

Model's specification

$$
	(1-\phi_1L-\phi_2 L^2)(1-L) y_t=u_t+\theta_1 u_{t-1}+\theta_2 u_{t-2}
$$

In [ ]:
forecaster = ARIMA(order=(2,1,2), trend='n')
forecaster.fit(y)
forecaster.summary()

In [ ]:
forecaster.get_fitted_params()

## Diagnostic of fitted model

Residuals (__please pay attention on missing values!__)

In [ ]:
forecaster.predict_residuals(y)

Graph of resudials

In [ ]:
forecaster.predict_residuals(y).plot()

plt.show()

Serial correlation test (Ljung-Box)

__Important__ we need to remove missing values in residuals

In [ ]:
# model_df = p+q
acorr_ljungbox(forecaster.predict_residuals(y).dropna() , lags=[7], model_df=2+2)

Heteroskedasticity test

__Important__ we need to remove missing values in residuals

In [ ]:
lm_stat, lm_pval, f_stat, f_pval = het_arch(forecaster.predict_residuals(y).dropna(), nlags=7, ddof=2+2)

lm_stat, lm_pval

## Forecasting

Let's forecast with ARIMA(1,1,1) with drift

Specification

$$
	(1-\phi L)(1-L) y_t=\alpha_0+u_t+\theta u_{t-1}
$$

Forecasting for 10 periods

In [ ]:
forecaster = ARIMA(order=(1,1,1), trend='n')
# forecasting horizon
fh = ForecastingHorizon(np.arange(1,11), freq ='W-Fri')

y_pred = forecaster.fit_predict(y=y, fh=fh)
y_pred

Forecasts' visuialization 

In [ ]:
plot_series(y.tail(50), y_pred, labels=['y', 'y_pred'])

plt.show()

Confidence interval

In [ ]:
conf_int = forecaster.predict_interval(fh=fh, coverage=0.9)
conf_int

In [ ]:
plot_series(y.tail(20), y_pred, labels=['y', 'y_pred'], markers=['o', 'X'], pred_interval=conf_int)

plt.show()

# Automatic order selection

Basic parameters of order selection

|Criterion|`AutoARIMA`|value|
|-|-|-|
|Information criterion|`information_criterion`|aic (default), aicc, bic, hqic |
|unit root test|`test`|kpss (default), adf, pp|
|max order d|`max_d`| 2 by default |
|max order p,q|`max_p`, ` max_q`|5 by default|
|seasonal or not|`seasonal`|True by default|
|significant level|`alpha`|0.05 by default|

__Remark__: order selection includes the best choice of drift/intercept/trend 

In [ ]:
forecaster = AutoARIMA(information_criterion='bic', test='kpss', seasonal=False)
forecaster.fit(y)
forecaster.get_fitted_params() #['order']